# Optimisation de patrimoine en 4 dimensions : rendement risque, risque extrême, éthique.



## Introduction :

Aujourd'hui, la gestion de patrimoine évolue selon les exigences croissantes des investisseurs. Ces dernier cherchent de plus en plus des approches plus éthiques et responsables dans leurs investissements. Dans un monde où les préoccupations financières traditionnelles se mêlent à la nécessité de faire face aux risques de tout genre, qu'ils soient extrêmes, climatiques, etc. notre projet vise à offrir une expertise transparente pour aider les épargnants à optimiser leur patrimoine. L'objectif principal est d'assurer leurs besoins futurs tout en intégrant des considérations éthiques, et ce, dans un contexte de transformation du monde par des financements "coup de cœur".

Le projet repose sur une approche en quatre dimensions, alliant rendement, risque, éthique, et une prévoyance spécifique face aux risques extrêmes. La dimension éthique introduit une perspective unique en évaluant chaque actif selon des critères éthiques spécifiques, permettant ainsi aux investisseurs de personnaliser leur portefeuille en fonction de leur valeurs.

### Étape 1 : Univers d'investissement éthique

La première étape consiste à construire un univers d'investissement éthique en évaluant chaque actif selon des critères éthiques définis. Ce processus implique une collaboration entre les étudiants pour attribuer une note éthique à chaque actif, créant ainsi un ensemble diversifié d'investissements alignés sur des valeurs partagées.

### Étape 2 : Optimisation en 3D

Dans cette phase, nous explorons différentes méthodes d'optimisation sous contraintes, intégrant des objectifs financiers et éthiques. En utilisant des cours historiques provenant de sources comme Yahoo Finance, nous testons des scénarios aléatoires de rendements. En investissant 10 000 €, une partie substantielle est protégée par des achats d'obligations d'État, tandis que le reste est réparti dans l'univers d'investissement défini. Nous évaluons également l'impact sur le portefeuille en introduisant des contraintes liées à l'inflation et à des notes éthiques minimales.

### Étape 3 : Optimisation en 4D

La dernière étape étend l'optimisation en intégrant la possibilité d'investir dans l'or et le Franc suisse pour atténuer les pertes en cas de krach. Des scénarios spécifiques définissent les variations des actifs pendant un krach, permettant une gestion proactive des risques extrêmes.

Ce projet s'inscrit dans une nouvelle ère de gestion de patrimoine, où les considérations éthiques, la gestion du risque et l'innovation financière convergent pour offrir une approche plus complète et personnalisée. En examinant ces quatre dimensions, nous aspirons à fournir une méthodologie robuste pour aider les investisseurs à naviguer dans un paysage financier complexe, tout en restant fidèles à leurs valeurs et en se préparant à des éventualités extraordinaires.

Ce notebook mêlera ébauches de code, ainsi que des conclusions quant à nos recherches et nos observations. Quant au travail réalisé, nous avons en grande partie réalisé le travail ensemble, mais concernant les grandes parties, Nassim s'est occupée de la partie sur les notations  ESG, et de la dernière partie sur la modélisation d'un krach. Quentin s'est quant à lui occupé de la partie optimisation ESG et donc des différentes méthodes.

## Étape 1

Pour construire notre univers d'investissement éthique nous nous sommes appuyer sur le site INVESTIR ETHIQUE. L'objectif de ce dernier est donner aux épargnants les clés pour faire fluctifier leur épargne, tout en respectant leurs convictions environnementales et sociales. Le site nous indique même que 60% des français sont intéressés par l'investissement responsable. Ce chiffre nous montre bien l'intérêt de traiter un tel sujet et la volonté des investisseurs à promouvoir les entreprises responsables, les fonds d'investissements verts, ou des entreprises proposant des projets d'énergies renouvelables.

Voici un aperçu du csv que nous avons créé en répertoriant les notations ESG des entreprises du CAC40. Le fichier csv classe dans l'ordre décroissant celles-ci selon leur note éthique finale, moyennant leurs notes dans les catégories Environnement, Social et Gouvernance.

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


notes_ethiques = pd.read_csv('note_éthique.csv')
scores_ethiques = notes_ethiques.set_index('Entreprise')['Score global']
notes_ethiques.head()

À présent grâce à la librairie python yfinance (Yahoo finance), nous allons récupérer les données historiques des différentes entreprises du CAC40

Pour rendre nos investissements et nos expérimentations plus simples, nous avons donc choisi de collecter les données depuis 2020 pour les 10 entreprises du CAC 40. On peut par ailleurs observer que le cours de l'action s'est considérérablement en plus de 10 ans.

In [ ]:
companies_names = ["Sanofi", "Air Liquide", "Thalès", "L'Oréal", "BNP", "Airbus", "Danone", "Axa", "Safran", "Total"]
companies_ticker = ["SAN.PA", "AI.PA", "HO.PA", "OR.PA", "BNP.PA", "AIR.PA", "BN.PA", "CS.PA", "SAF.PA", "TTE.PA"]

prices = yf.download(companies_ticker, start="2020-01-31", end="2024-01-15")["Close"]

# Converting datetime format
prices.reset_index(inplace=True)
prices['Date'] = prices['Date'].dt.strftime('%Y/%m/%d')
prices['Date'] = pd.to_datetime(prices['Date'])
prices.set_index('Date', inplace=True)

prices.head()

In [ ]:
# On interpole les données manquantes
prices = prices.interpolate(method="nearest")
prices.tail().round(1)

# On calcule les rendements
sample_returns = prices.pct_change().dropna()
sample_returns

In [ ]:
sample_ethic_scores = notes_ethiques[notes_ethiques['Entreprise'].isin(companies_names)].set_index('Entreprise').reindex(companies_names)

sample_scores = sample_ethic_scores["Score global"]
print(sample_scores)

Nous allons travailler avec un dictionaire réportoriants les différentes dimensions qui nous servirons pour construire notre portefeuille optimal, à savoir le risque ( ou l'écart-type des returns ), la rentabilité ( l'espérances de ces derniers) ainsi que le score ESG. On va travailler avec les rentabilité, volatilitées annuelles. Voici ce à quoi vont ressembler nos rentabilitées annuelles.

In [ ]:
# Risk-Return profile
ret_stats = sample_returns.agg(['mean','std']).T
ret_stats.columns=['Return','Risk']
ret_stats['Return'] = ret_stats['Return']*252
ret_stats['Risk'] = ret_stats['Risk']*np.sqrt(252)
ret_stats['Sharpe_Ratio']=(ret_stats['Return']-0.01)/ret_stats['Risk'] # rf: US 10y Bond Yield at January 31,2023
ret_stats = ret_stats.sort_values('Sharpe_Ratio', ascending=False)
ret_stats.round(3)

In [ ]:
# Plotting annualized Risk, Return and Sharpe Ratio
from adjustText import adjust_text
fig = plt.figure(figsize=(10,5))
plt.scatter(data=ret_stats,x='Risk',y='Return', c='Sharpe_Ratio', cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
texts = []
for x, y, s in zip(ret_stats['Risk'],ret_stats['Return'], ret_stats.index):
    texts.append(plt.text(x,y,s))
adjust_text(texts)
plt.xlabel('Annualized Risk'), plt.ylabel('Annualized Return')
plt.title('Risk-Return profile', weight='bold')

Le ratio de Sharpe se définit comme le rendement excédentaire des actions (ou du portefeuille) divisé par l'écart type des rendements. On remarque facilement que L'Oréal a le meilleur ratio. On le remarque également facilement dans la graphique ci-dessus, car il s'agit du stock avec le rendement le plus pour un niveau de risque faible.

Étant donné que tous les stocks ont un bêta inférieur à un, nous pouvons dire qu'ils ont tous une volatilité inférieure à celle du marché, c'est-à-dire que l'inclusion de chacun d'entre eux dans un portefeuille le rend moins risqué que le même portefeuille sans cette action.

De même, à titre d'indication pour la suite, nous pouvons nous intéresser aux rendements journaliers cumulatifs

In [ ]:
# Calculation of cumulative returns, as the growth of £1 
cumulative_returns = ((1 + sample_returns).cumprod()-1)*100

# Rates of change for each company share
ticker_change = cumulative_returns.tail(1).round(3)
ticker_change

In [ ]:
def line_plot(data,legend_loc, legend_ncol, title, ylabel):
    plt.figure(figsize=(15,5))
    for col in data:
        plt.plot(data.index, data[col], marker='', linewidth=1, label=col)
        plt.legend(loc=legend_loc, ncol=legend_ncol)
        plt.margins(x=0)
        plt.title(title, weight='bold')
        plt.ylabel(ylabel)
    plt.show()
    
# Plotting daily cummulative returns
line_plot(data=cumulative_returns, legend_loc='upper left', legend_ncol=1,
            title='Daily Cummulative Returns', ylabel='Cummulative Return')


On peut dès lors observer en 2020 un creux pour l'ensemble des stocks de notre échantillon, dû à la crise du covid, notamment Safran ou Airbus qui ont vu leur production particulièrement impactée par les confinements successifs. L'Oréal est l'entreprise qui a eu la tendance la plus haussières ces 10 dernières années et qui demeure être celle avec les plus gros rendements aujourd'hui.

In [ ]:
mean_ret = sample_returns.mean()*252
cov_matrix = sample_returns.cov(numeric_only=True)*252
cov_matrix


Nous allons utiliser la fonction ci-dessous pour générer aléatoirement les poids de notre portefeuille. Cette méthode permet de créer des vecteurs de poids contenant parfois des 0, ou des répartitions plus aléatoires que la fonction native random.

In [ ]:
def random_weights_for_portfolio(stocks_number):
    zero_vector = np.zeros(stocks_number)
    random_weights_vector = zero_vector.copy()
    num_zeros = np.random.randint(0, len(zero_vector) + 1)
    
    if num_zeros < len(zero_vector):
        random_indices = np.random.choice(len(zero_vector), len(zero_vector) - num_zeros, replace=False)
        for index in random_indices:
            random_weights_vector[index] = np.random.random()
    
    # Normalize the weights so that their sum is equal to 1
    total = np.sum(random_weights_vector)
    if total != 0:
        random_weights_vector = random_weights_vector / total
    else:
        random_weights_vector = np.full_like(random_weights_vector, 1 / len(random_weights_vector))
    
    return random_weights_vector


In [ ]:
std_returns = sample_returns.std()*np.sqrt(252)

assets_num=len(companies_names)
port_num=200000
port_ret=[]
port_vol=[]
port_ethic_score = []
port_weights=[]
sharpe_ratio=[]

for portfolio in range(port_num):
    weights = random_weights_for_portfolio(assets_num)
    ret = np.dot(weights, mean_ret)
    vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix,weights)))
    ethic = np.dot(weights, sample_scores)
    rf = 0.01
    sharpe = (ret-rf)/vol
    sharpe_ratio.append(sharpe)
    port_ret.append(ret)
    port_vol.append(vol)
    port_weights.append(weights)
    port_ethic_score.append(ethic)
    
# Dictionnaires contenant les données de chaque portefeuilles
port_dict = {'Returns': port_ret,
             'Volatility': port_vol, 
             'Sharpe Ratio': sharpe_ratio, 
             'Ethic Score' : port_ethic_score}

for counter,symbol in enumerate(companies_ticker):
    port_dict[symbol] = [Weight[counter] for Weight in port_weights]

port_EF = pd.DataFrame(port_dict)

gmv_port_value = port_EF['Volatility'].min()
msr_port_value = port_EF['Sharpe Ratio'].max()

gmv_port = port_EF.loc[port_EF['Volatility'] == gmv_port_value]
msr_port = port_EF.loc[port_EF['Sharpe Ratio'] == msr_port_value]

port_EF.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio', cmap='viridis', edgecolors='black', figsize=(10, 8), grid=True)
plt.xlabel('Volatility (Standard Deviation)')
plt.ylabel('Expected Returns')
plt.title('Portfolio Optimization: Efficient Frontier', weight='bold')
plt.scatter(x=gmv_port['Volatility'], y=gmv_port['Returns'], c='blue', marker='*', s=500, label='Global Minimum Variance')
plt.scatter(x=msr_port['Volatility'], y=msr_port['Returns'], c='red', marker='*', s=500, label='Maximum Sharpe Ratio')
plt.legend(labelspacing=1.2)
plt.grid(True)
plt.show()

Ci-dessus nous avons donc simuler aléatoirement un grand nombre de portefeuilles pour créér un graphique 2D mettant en relation risque et rentabilitées. Par ailleurs nous avons affiché en bleu le portefeuille de variance (risque) minimale et en rouge le portfeuille des ratio de Sharpe minimal dont voici les poids : 

In [ ]:
msr_port.round(3)

Maintenant que nous avons tracé notre graphique en 2 dimensions (seulement avec le risque et les rentabilités). Tentons d'intégrer une nouvelle dimension: le score ESG de ces portefeuilles, obtenus en multipliant les poids par les scores respectifs de chaque entreprise.

In [ ]:
import plotly.express as px

threshold = 0.85

In [ ]:
fig = px.scatter_3d(port_dict, x='Volatility', y='Returns', z='Ethic Score', color='Sharpe Ratio')
fig.add_scatter3d(x=port_vol, y=port_ret, z=threshold*np.ones_like(port_vol), mode='lines', name='Threshold Plane', line=dict(color='black', width=2))

fig.update_traces(marker=dict(size=5, opacity=0.8), selector=dict(mode='markers')) 
fig.update_layout(scene=dict(xaxis_title='Volatility', yaxis_title='Returns', zaxis_title='Ethic Score'))  
fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.update_layout(title='3D Scatter Plot', title_x=0.5) 

fig.show()

In [ ]:
plt.scatter(port_EF["Volatility"][port_EF["Ethic Score"] > threshold], port_EF["Returns"][port_EF["Ethic Score"] > threshold], c=port_EF["Sharpe Ratio"][port_EF["Ethic Score"] > threshold], s=20, cmap='viridis')

for i in range(len(companies_names)):
    plt.scatter(std_returns[i], mean_ret[i], s=100, marker='o', label=companies_names[i])
    
plt.xlabel('Volatility (Standard Deviation)')
plt.ylabel('Expected Returns')
plt.title('Portfolio above the threshold', weight='bold')
plt.legend(labelspacing=1.2)
plt.grid(True)
plt.show()

Ci-dessus, nous avons uniquement affiché les portefeuilles étant au dessus du seuil accordé aux entreprises en terme d'ESG. On remarque que le nombre de portefeuilles réalisables diminue considérablement. Nous pouvons des à présent afficher celui qui maximise le ratio de sharpe.

In [ ]:
port_argmax_sr = port_EF["Sharpe Ratio"].argmax()
port_max_sr = port_EF["Sharpe Ratio"].max()
port_argmax_sr_w_esg = port_EF["Sharpe Ratio"][port_EF["Ethic Score"] > threshold].argmax()
port_max_sr_w_esg = port_EF["Sharpe Ratio"][port_EF["Ethic Score"] > threshold].max()

print(f"Portefeuille maximisant le ratio de Sharpe (sans seuil ESG): \n {port_EF.loc[port_EF['Sharpe Ratio'] == port_max_sr].iloc[:, 4:]}")
weights1 = port_EF.loc[port_EF['Sharpe Ratio'] == port_max_sr].iloc[:, 4:]
print(f"Rentabilité : {port_ret[port_argmax_sr]}")
print(f"Volatilité : {port_vol[port_argmax_sr]}")
print(f"Ratio de Sharpe : {port_max_sr}")
print(f"Score ESG : {port_ethic_score[port_argmax_sr]}")
print("\n")

print(f"Portefeuille maximisant le ratio de Sharpe (avec seuil ESG): \n {port_EF.loc[port_EF['Sharpe Ratio'] == port_max_sr_w_esg].iloc[:, 4:]}")
weights1_w_esg = port_EF.loc[port_EF['Sharpe Ratio'] == port_max_sr_w_esg].iloc[:, 4:]
print(f"Rentabilité : {port_ret[port_argmax_sr_w_esg]}")
print(f"Volatilité : {port_vol[port_argmax_sr_w_esg]}")
print(f"Ratio de Sharpe : {port_max_sr_w_esg}")
print(f"Score ESG : {port_ethic_score[port_argmax_sr_w_esg]}")

On remarque donc bien une différence de composition du portefeuille si l'on considère ou non les actifs avec des bonnes notes ESG. On voit également que la rentabilité du portefeuille diminue si le score global ESG augmente.

### Méthode 2: Scipy optimizer and Bonus ESG

L'idée ici est d'ajouter un bonus de rentabilité aux entreprises plus responsable et ayant un meilleurs score ESG. Cette manipulation est assez simple à mettre en place et permet de directement choisir ces investissement et son portefeuille avec 2 dimensions au final. Nous allons également utiliser l'optimizer natif de scipy pour optimiser nos poids de notre portefeuille. 
En premier lieu, il s'agit de déterminer quel bonus accorder à ces rentabilitées pour ne pas biaiser complètement nos résultats. Par exemple, accordons un bonus de rentabilité de 0.001 à toutes les entreprises ayant un score ESG supérieur ou égale à 0,6. 

In [ ]:
bonus = 0.011

temp_mean_ret = mean_ret
for i in range(len(sample_scores)):
    if sample_scores[i] > 0.6: 
        temp_mean_ret[i] += bonus


In [ ]:
def portfolio_return(weights, mean_ret): 
    return np.dot(weights, mean_ret)

def portfolio_vol(weights, covmat):
    return np.sqrt(np.dot(weights.T, np.dot(covmat,weights)))


In [ ]:
from scipy.optimize import minimize

def minimize_vol(target_return, er, cov):
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n
    return_is_target = {
        'type' : 'eq',
        'args' : (er,),
        'fun' : lambda weights, er : target_return - portfolio_return(weights, er)
    }
    weights_sum_to_1 = {
        'type' : 'eq',
        'fun' : lambda weights : np.sum(weights) - 1
    }
    results = minimize(portfolio_vol, init_guess,
                       args = (cov,), method = 'SLSQP',
                       options={'disp': False},
                       constraints= (return_is_target, weights_sum_to_1),
                       bounds = bounds
                      )
    return results.x

def optimal_weights(n_points, er, cov):
    target_rs = np.linspace(er.min(), er.max(), n_points)
    weights = [minimize_vol(target_return, er, cov) for target_return in target_rs]
    return weights

def plot_ef(n_points, er, cov):
    weights = optimal_weights(n_points, er, cov)
    rets = [portfolio_return(w, er) for w in weights]
    vols = np.array([portfolio_vol(w, cov) for w in weights])
    ef = pd.DataFrame({'Returns': rets, 'Volatility': vols})
    fig = ef.plot(x='Volatility', y='Returns', style='.-', color='green')
    
    return fig

plot_ef(1000, temp_mean_ret, cov_matrix)

L'objectif à présent est déterminer le portefeuille avec le ratio de Sharpe maximum, en traçant la ligne de marché (capital market line), qui est la droite tangente tracée depuis le point de l'actif sans risque jusqu'à la région réalisable pour les actifs risqués. Le point de tangence est le portefeuille avec le ratio de Sharpe maximal.

In [ ]:
def msr(risk_free_rate, er, cov):
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n
    weights_sum_to_1 = {
        'type' : 'eq',
        'fun' : lambda weights : np.sum(weights) - 1
    }
    
    def neg_sharpe_ratio(weights, risk_free_rate, er, cov):
        '''
        Returns the negative sharpe ratio, given weights
        '''
        r = portfolio_return(weights, er)
        vol = portfolio_vol(weights, cov)
        return -(r - risk_free_rate)/vol
    
    results = minimize(neg_sharpe_ratio, init_guess,
                       args = (risk_free_rate, er,cov,), method = 'SLSQP',
                       options={'disp': False},
                       constraints= (weights_sum_to_1),
                        bounds = bounds
                       )
    return results.x

In [ ]:
w_msr = msr(0.1, temp_mean_ret, cov_matrix)
ret_msr = portfolio_return(w_msr, temp_mean_ret)
vol_msr = portfolio_vol(w_msr, cov_matrix)
ax = plot_ef(1000, temp_mean_ret, cov_matrix)

# Adding CML 

cml_x = [0, vol_msr]
cml_y = [0.1, ret_msr]
ax.plot(cml_x, cml_y, color='goldenrod', marker= 'x', markersize= 10)
ax.set_xlim(left = 0)
ax.set_title('Efficient Frontier and CML')

In [ ]:
print(f"Rentabilité et volatilité du portefeuille max ratio de sharpe : {ret_msr, vol_msr}")

In [ ]:
def format_decimal(value):
    return float('{:.6f}'.format(value))

weights_bonus = pd.DataFrame({'Weights bonus': w_msr}, index=sample_returns.columns).sort_values(by='Weights bonus')[:]
weights_bonus = weights_bonus.applymap(format_decimal)
weights_bonus

On peut remarquer en conclusion que nous avons un portefeuille composé majoritaiement de quelques actions, à savoir L'Oréal, Safran ou encore Thalès, 2 entreprises ayant des excellentes notes ESG et de bonne rentabilités. De même, Thales, qui a une rentabilité égale à celle de L'Oréal à une place bien moins importante que celle-ci dans notre portefeuille, dûe à une note ESG faible.

Dans le même principe et pour étendre nos méthode, nous pourrions mettre en place un système de malus pour les entreprises qui possèdent des notes ESG faibles.

In [ ]:
malus = -0.01

temp_mean_ret_malus = mean_ret
for i in range(len(sample_scores)):
    if sample_scores[i] < 0.5: 
        temp_mean_ret_malus[i] += malus

In [ ]:
w_msr = msr(0.1, temp_mean_ret_malus, cov_matrix)
ret_msr = portfolio_return(w_msr, temp_mean_ret)
vol_msr = portfolio_vol(w_msr, cov_matrix)
ax = plot_ef(1000, temp_mean_ret_malus, cov_matrix)

# Adding CML 

cml_x = [0, vol_msr]
cml_y = [0.1, ret_msr]
ax.plot(cml_x, cml_y, color='goldenrod', marker= 'x', markersize= 10)
ax.set_xlim(left = 0)
ax.set_title('Efficient Frontier and CML')

In [ ]:
weights_malus = pd.DataFrame({'Weights malus': w_msr}, index=sample_returns.columns).sort_values(by='Weights malus')[:]
weights_malus = weights_malus.applymap(format_decimal)
weights_malus

On remarque dès lors que Thalès ayant une très bonne rentabilité n'apparait quasiment plus dans notre portefeuille. L'Oréal, ayant de bons rendements et une note éthique très haute devient majoritaire dans notre portefeuille.

### Méthode 3 : Optimisateur Scipy et contrainte ESG

Cette méthode va consister à simplement utiliser l'optimisateur de Scipy en ajoutant une contrainte ESG, par exemple une note ESG égale à 0.8, et minimiser la volatilité de la même manière que dans la méthode précédente

In [ ]:
def minimize_vol_w_esg(target_return, er, cov, esg_score, threshold):
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n
    return_is_target = {
        'type' : 'eq',
        'args' : (er,),
        'fun' : lambda weights, er : target_return - portfolio_return(weights, er)
    }
    weights_sum_to_1 = {
        'type' : 'eq',
        'fun' : lambda weights : np.sum(weights) - 1
    }
    esg_score_to_threshold = {
        'type' : 'eq',
        'args' : (esg_score,),
        'fun' : lambda weights, esg_score : portfolio_return(weights, esg_score) - threshold
    }
    results = minimize(portfolio_vol, init_guess,
                       args = (cov,), method = 'SLSQP',
                       options={'disp': False},
                       constraints= (return_is_target, weights_sum_to_1, esg_score_to_threshold),
                       bounds = bounds
                      )
    return results.x

def optimal_weights_w_esg(n_points, er, cov, esg_score, threshold):
    target_rs = np.linspace(er.min(), er.max(), n_points)
    weights = [minimize_vol_w_esg(target_return, er, cov, esg_score, threshold) for target_return in target_rs]
    return weights

def plot_ef_w_esg(n_points, er, cov, esg_score):
    threshold = 0.6
    
    weights_w_esg = optimal_weights_w_esg(n_points, er, cov, esg_score, threshold)
    rets_w_esg = [portfolio_return(w, er) for w in weights_w_esg]
    vols_w_esg = np.array([portfolio_vol(w, cov) for w in weights_w_esg])
    
    weights = optimal_weights(n_points, er, cov)
    rets = [portfolio_return(w, er) for w in weights]
    vols = np.array([portfolio_vol(w, cov) for w in weights])
    
    ef = pd.DataFrame({'Returns': rets, 'Volatility': vols})
    ef_w_esg = pd.DataFrame({'Returns with ESG': rets_w_esg, 'Volatility with ESG': vols_w_esg})

    fig = ef.plot(x='Volatility', y='Returns', style='.-', color='green')
    ef_w_esg.plot(x='Volatility with ESG', y='Returns with ESG', style='.-', color='blue', ax=fig)
    
    return fig

plot_ef_w_esg(1000, mean_ret, cov_matrix, sample_scores)

In [ ]:
def msr_w_esg(risk_free_rate, er, cov, esg_score, threshold):
    weights_w_esg = optimal_weights_w_esg(1000, er, cov, esg_score, threshold)
    rets_w_esg = [portfolio_return(w, er) for w in weights_w_esg]
    vols_w_esg = np.array([portfolio_vol(w, cov) for w in weights_w_esg])
    sr = []
    risk_free_rate = 0.1
    for ret, vol in zip(rets_w_esg, vols_w_esg):
        sr.append((ret - risk_free_rate)/vol)
    index_max_rs = sr.index(max(sr))
    return weights_w_esg[index_max_rs]


In [ ]:
w_msr_esg = msr_w_esg(0.1, mean_ret, cov_matrix, sample_scores, 0.6)
weights_esg_constraint = pd.DataFrame({'Weights ESG constraint': w_msr_esg}, index=sample_returns.columns).sort_values(by='Weights ESG constraint')[:]
weights_esg_constraint = weights_esg_constraint.applymap(format_decimal)
weights_esg_constraint

In [ ]:
ret_msr_esg = portfolio_return(w_msr_esg, mean_ret)
vol_msr_esg = portfolio_vol(w_msr_esg, cov_matrix)
ax = plot_ef_w_esg(1000, mean_ret, cov_matrix, sample_scores)

# Adding CML 

cml_x = [0, vol_msr_esg]
cml_y = [0.1, ret_msr_esg]
ax.plot(cml_x, cml_y, color='goldenrod', marker= 'x', markersize= 10)
ax.set_xlim(left = 0)
ax.set_title('Efficient Frontier and CML')

Dans le même contexte, plutôt que de fixer une contrainte seuil pour notre score ESG, nous pourrions créer une fonction d'utilité à minimiser, dans le but de maximiser le score ESG tout en minimisant la volatilité à rentabilité égale.

In [ ]:
def utility_function(weights, cov, esg_score):
    portfolio_volatility = portfolio_vol(weights, cov)
    portfolio_esg_score = portfolio_return(weights, esg_score)

    return portfolio_volatility - portfolio_esg_score

In [ ]:
def minimize_vol_max_esg(target_return, er, cov, esg_score):
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n

    utility_loss = lambda weights: utility_function(weights, cov, esg_score)

    weights_sum_to_1 = {
        'type' : 'eq',
        'fun' : lambda weights : np.sum(weights) - 1
    }
    
    return_is_target = {
        'type' : 'eq',
        'args' : (er,),
        'fun' : lambda weights, er : target_return - portfolio_return(weights, er)
    }

    results = minimize(utility_loss, init_guess,
                       method = 'SLSQP',
                       options={'disp': False},
                       constraints= (weights_sum_to_1, return_is_target),
                       bounds = bounds
                      )
    return results.x

On tente donc ici dans la fonction minimize_vol_max_esg de minimiser notre fonction utilité précédente.

In [ ]:
def optimal_weights_max_esg(n_points, er, cov, esg_score):
    target_rs = np.linspace(er.min(), er.max(), n_points)
    weights = [minimize_vol_max_esg(target_return, er, cov, esg_score) for target_return in target_rs]
    return weights

def plot_ef_max_esg(n_points, er, cov, esg_score):    
    weights_max_esg = optimal_weights_max_esg(n_points, er, cov, esg_score)
    rets_w_esg = [portfolio_return(w, er) for w in weights_max_esg]
    vols_w_esg = np.array([portfolio_vol(w, cov) for w in weights_max_esg])
    
    weights = optimal_weights(n_points, er, cov)
    rets = [portfolio_return(w, er) for w in weights]
    vols = np.array([portfolio_vol(w, cov) for w in weights])
    
    ef = pd.DataFrame({'Returns': rets, 'Volatility': vols})
    ef_w_esg = pd.DataFrame({'Returns with ESG': rets_w_esg, 'Volatility with ESG': vols_w_esg})

    fig = ef.plot(x='Volatility', y='Returns', style='.-', color='green')
    ef_w_esg.plot(x='Volatility with ESG', y='Returns with ESG', style='.-', color='blue', ax=fig)
    
    return fig

plot_ef_max_esg(1000, mean_ret, cov_matrix, sample_scores)

In [ ]:
def msr_max_esg(risk_free_rate, er, cov, esg_score, threshold):
    weights_max_esg = optimal_weights_max_esg(1000, er, cov, esg_score)
    rets_max_esg = [portfolio_return(w, er) for w in weights_max_esg]
    vols_max_esg = np.array([portfolio_vol(w, cov) for w in weights_max_esg])
    sr = []
    risk_free_rate = 0.1
    for ret, vol in zip(rets_max_esg, vols_max_esg):
        sr.append((ret - risk_free_rate)/vol)
    index_max_rs = sr.index(max(sr))
    return weights_max_esg[index_max_rs], max(sr)


w_msr_max_esg = msr_max_esg(0.1, mean_ret, cov_matrix, sample_scores, 0.6)[0]
weights_max_esg_constraint = pd.DataFrame({'Weights ESG constraint': w_msr_max_esg}, index=sample_returns.columns).sort_values(by='Weights ESG constraint')[:]
weights_max_esg_constraint = weights_max_esg_constraint.applymap(format_decimal)
weights_max_esg_constraint

Pour conclure sur cette méthode, au vu de la frontière nous ne sommes pas sûr de la cohérence des résultats.

### Conclusion

Observons de manière plus illustrative, nous allons comparer nos différentes méthodes.

In [ ]:
# Portolio weights
msr = msr_port.iloc[:, 4:]

def transform_df(df, name): 
    df_stacked = df.reset_index().melt(id_vars='index', var_name='Ticker', value_name=name)
    df_stacked = df_stacked.drop('index', axis=1).set_index('Ticker')
    return pd.DataFrame(df_stacked)



msr_weights = transform_df(msr, 'Weights MSR')
weights_method1 = transform_df(weights1, 'Weights Method1')
weights_method1_w_esg = transform_df(weights1_w_esg, 'Weights Method1 esg')


weights_df = pd.concat([msr_weights, weights_method1, weights_method1_w_esg, weights_bonus, weights_malus, weights_esg_constraint],axis=1)

fig = plt.figure(figsize=(20,10))
colours = dict(zip(weights_df.index, plt.cm.tab10.colors[:len(weights_df.index)]))
for column,num in zip(weights_df.columns, range(len(weights_df.columns))):
    plt.subplot(1,6,num+1)
    labels, values = zip(*((key,value) for key,value in weights_df[column].items() if value>0))
    plt.pie(values, labels=labels, colors=[colours[key] for key in labels], autopct='%1.1f%%')
    plt.title('{} Portfolio'.format(column), weight='bold')
    fig.tight_layout()
plt.show()

Pour conclure avec cette partie "optimisation en 3D", nous allons choisir une des méthodes que nous avons évoquée précédemment et la mettre en application dans le cadre construire un portefeuille.
Maintenant, nous considérons que nous investissons 10 000€.

In [ ]:
# Investment amount
total_investment = 10000
bond_allocation_percentage = 0.7
bond_annual_return = 0.1

# Allocate a fraction of the investment to bonds
bond_investment = total_investment * bond_allocation_percentage

# Calculate the remaining amount for stock investment
stock_investment = total_investment - bond_investment


Si l'on choisit de prendre le portefeuille avec une note ESG minimale de 0.6 (méthode avec scipy et la contrainte ESG), on obtiendra alors un portefeuille de la valeur ci-dessous.

In [ ]:
bond_investment_value = bond_investment * (1 + bond_annual_return)

weights_transposed = weights_esg_constraint.transpose()
stock_investment_value = (weights_transposed * stock_investment).sum()

total_investment_value = bond_investment_value + stock_investment_value

print("Valeur totale de l'investissement après un an :\n", total_investment_value)

### Gestion indicielle et inflation

Notre objectif va maintenant être de tenter de suivre l'inflation. C'est-à-dire de non-plus construire un portefeuille qui minimise la volatilité ou maximise la rentabilité, mais cette fois de construire un portefeuille qui minimise la tracking error par rapport à notre benchmark, qui est ici l'inflation. Notre idée va être de tester nos deux types de portfeuilles, avec ou sans contrainte ESG. Dans un premier il s'agit de définir notre fonction à minimiser. On sait que pour une gestion indicielle, notre système à optimiser a la forme :

\begin{cases}
\min_{x_{t}} ((x_{t} - C)^{T}V(x_{t} - C)) \\
x_{t}^{T} 1_{N} = 1
\end{cases}

avec C composition de l'actif et V la matrice de covariance des rentabilités

Pour suivre l'inflation nous avons trouvé un ETF qui suit l'inflation en zone euro (on part cette fois-ci de 2021): 

In [ ]:
etf_inflation = yf.download('0WAP.L', start="2021-01-31", end="2024-01-15")['Close']
etf_inflation = etf_inflation.interpolate(method="nearest")
etf_inflation.tail().round(1)

# On calcule les rendements
inflation_returns = etf_inflation.pct_change().dropna()

returns_from_2021 = sample_returns.loc['2021-01-31':]

plt.figure(figsize=(12, 6))
inflation_returns.plot(color='blue', lw=1, label='ETF sur l\'inflation (0WAP.L)')
plt.title('Performance de l\'ETF sur l\'inflation')
returns_from_2021.plot(color='blue', lw=1)
plt.xlabel('Date')
plt.ylabel('Prix de clôture')
plt.grid(True)
plt.show()

On peut remarquer au vu du tracé quelques similarités notamment au niveaux des pics. Par la suite créons les différents vecteurs squi nous servirons à optimiser notre portefeuille.

In [ ]:
#rentas annuelles de l'inflation
mean_ret_inflation = inflation_returns.mean()*252 

#Matrice C 
C = np.zeros(len(mean_ret) + 1)
C[-1] = 1

# vecteur des rentabilités 
returns_w_inflation = {ticker: round(value, 5) for ticker, value in mean_ret.items()}
returns_w_inflation['Inflation'] = round(mean_ret_inflation,5)
returns_w_inflation = pd.Series(returns_w_inflation, name='Rentabilités')


# poids portefeuille
inflation_row = pd.DataFrame({'Weights MSR': [0.0]}, index=['Inflation'])
X = msr_weights
X = X.append(inflation_row)

# matrice covariance 
returns_df = returns_from_2021
inflation_returns = pd.DataFrame(inflation_returns)
inflation_returns = inflation_returns.rename(columns={'Close': 'Inflation'})

returns_df = pd.concat([returns_from_2021, inflation_returns],axis=1).dropna()
cov_matrix_inflation = returns_df.cov(numeric_only=True)*252

In [ ]:
def tracking_error(weights, cov):
    x_minus_C_transpose = (weights - C).reshape(1, -1)
    return np.dot(x_minus_C_transpose, np.dot(cov, (weights - C)))
    
def minimize_vol_inflation(er, cov):
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    init_guess[-1] = 0.0

    bounds = ((0.0, 1.0),) * (n - 1) + ((0.0, 0.0),)
    weights_sum_to_1 = {
        'type' : 'eq',
        'fun' : lambda weights : np.sum(weights) - 1
    }
    results = minimize(tracking_error, init_guess,
                       args = (cov,), method = 'SLSQP',
                       options={'disp': False},
                       constraints= (weights_sum_to_1),
                       bounds = bounds
                      )
    return results.x

def optimal_weights_w_inflation(er, cov):
    weights = minimize_vol_inflation(er, cov)
    return weights

weights_inflation = optimal_weights_w_inflation(returns_w_inflation, cov_matrix_inflation)
weights_inflation

In [ ]:
ret = portfolio_return(weights_inflation, returns_w_inflation)
vol = portfolio_return(weights_inflation, cov_matrix_inflation)

weights_inflation_benchmark = pd.DataFrame({'Weights inflation benchmark': weights_inflation}, index=returns_df.columns).sort_values(by='Weights inflation benchmark')[:]
weights_inflation_benchmark = weights_inflation_benchmark.applymap(format_decimal)

weights_inflation_benchmark

## Optimisation en 4D

Nous allons à présent inclure dans nos investissements la possibilité d'investir dans de l'or et du franc suisse. L'idée est de concevoir une situation de Krach boursier et d'optimiser notre portefeuille pour répondre au mieux à cette inadvertance. Dans un premier temps simuler de la manière la plus simple possible un scénario de krach aléatoirement sur nos 10 stocks et tentons d'optimiser notre portefeuille. 

In [ ]:
 # Définition des rendements en cas de krach pour chaque action
krach_returns_data = {
    "SAN.PA": -0.2, 
    "AI.PA": -0.3,  
    "HO.PA": -0.3,  
    "OR.PA": -0.4, 
    "BNP.PA": -0.4, 
    "AIR.PA": -0.2, 
    "BN.PA": -0.3, 
    "CS.PA": -0.4, 
    "SAF.PA": -0.4,  
    "TTE.PA": -0.2,  
}

In [ ]:
# Calcul de la perte potentielle en cas de krach
def krach_loss(weights, krach_returns):
    krach_portfolio_ret = np.dot(weights, list(krach_returns.values()))
    loss = -krach_portfolio_ret
    return loss

def optimize_portfolio_krach(krach_returns):
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(len(krach_returns)))
    init_weights = [1/len(krach_returns)] * len(krach_returns)
    optimal_weights = minimize(krach_loss, init_weights, args = (krach_returns,), method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x

optimal_weights = optimize_portfolio_krach(krach_returns_data)
print("Pondérations optimales du portefeuille pour minimiser la perte en cas de krach :")
for company, weight in zip(companies_names, optimal_weights):
    print(f"{company}: {weight:.4f}")

Sans contrainte ESG et après le krach, on peut voir que notre portefeuille est constitué majoritairement des stocks ayant été les moins exposés aux baisses des rentabilités.

In [ ]:
def optimize_portfolio_krach_w_esg(esg_score, krach_returns):
    threshold = 0.7
    weights_constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    esg_score_to_threshold = {
        'type' : 'eq',
        'args' : (esg_score,),
        'fun' : lambda weights, esg_score : portfolio_return(weights, esg_score) - threshold
    }
    bounds = tuple((0, 1) for _ in range(len(krach_returns)))
    init_weights = [1/len(krach_returns)] * len(krach_returns)
    optimal_weights = minimize(krach_loss, init_weights, args = (krach_returns,), method='SLSQP', bounds=bounds, constraints=(weights_constraints, esg_score_to_threshold))
    return optimal_weights.x

optimal_weights_w_esg = optimize_portfolio_krach_w_esg(sample_scores, krach_returns_data)
print("Pondérations optimales du portefeuille pour minimiser la perte en cas de krach avec contraintes ESG:")
for company, weight in zip(companies_names, optimal_weights_w_esg):
    print(f"{company}: {weight:.4f}")

On remarque que notre portefeuille est cette fois-ci constituer de seulement 2 actifs, L'Oréal (pas présent dans le portefeuille précédent) qui a une note éthique très élevée, et Airbus, qui voit son rendement diminuer de seulement 0.2 et que a une note éthique plutôt convenable.

Nous allons maintenant illustrer ces krach boursiers de manière plus réaliste. En effet, plutôt que de diminuer nos rentabilités annuelles moyennes directement, ce qui semble demesuré. Nous allons insérer dans notre dataframe des rendements quelques lignes (plusieurs dates futures simulées), qui vont illustrer un krach.

In [ ]:
# Création de 2 nouvelles dates où le krach apparait
future_dates = pd.date_range(start='2024-01-13', periods=2, freq='D')  # Nouvelles dates

np.random.seed(42)
krach_returns = pd.DataFrame(np.random.uniform(-0.2, -0.3, size=(2, 10)), columns=sample_returns.keys(), index=future_dates)

data_with_krach = pd.concat([pd.DataFrame(sample_returns), krach_returns])
data_with_krach.tail()

In [ ]:
mean_ret_krach = data_with_krach.mean()*252
mean_ret_krach_dict = {ticker: round(value, 4) for ticker, value in mean_ret_krach.items()}

optimal_weights_w_esg = optimize_portfolio_krach_w_esg(sample_scores, mean_ret_krach_dict)
print("Pondérations optimales du portefeuille pour minimiser la perte en cas de krach avec contraintes ESG:")
for company, weight in zip(companies_names, optimal_weights_w_esg):
    print(f"{company}: {weight:.4f}")

On retrouve encore une fois après le krach et sous contrainte ESG un portefeuille constitué de L'Oréal en majorité, du fait de son score éthique élevé. Cependant si l'on refait notre simulation avec des données de krach différentes (seed qui change), on s'aperçoit que notre portefeuille a composition bien différente, avec toujours L'Oréal, mais également les stocks qui ont été le moins impacté par le krach et ayant des scores ESG au dessus du threshold fixé lors de l'optimisation.

Dorénavant nous allons ajouté la possibilité d'investir dans de l'or ou du franc suisse. Nous allons donc récupérer leurs données historiques grâce à yfinance.

In [ ]:
gold_data = yf.download('GC=F', start="2020-01-31", end="2024-01-15")['Close']

# On interpole les données manquantes
gold_data = gold_data.interpolate(method="nearest")
gold_data.tail().round(1)

# On calcule les rendements de l'or
sample_returns_gold = gold_data.pct_change().dropna()
sample_returns_gold = pd.DataFrame(sample_returns_gold)
sample_returns_gold = sample_returns_gold.rename(columns={'Close': 'Gold'})

sample_returns_w_gold = pd.concat([sample_returns, sample_returns_gold],axis=1)
sample_returns_w_gold

Une des premières étapes consiste à ajouter une note ESG à notre or. Nous allons choisir la note par défaut de 0.0, car cet actif ne rentre dans aucun critère ESG.

In [ ]:
sample_scores_w_gold_dict = {company: round(value, 5) for company, value in sample_scores.items()}
sample_scores_w_gold_dict['Gold'] = 0.0
sample_scores_w_gold = pd.Series(sample_scores_w_gold_dict, name='Score global')
companies_names.append('gold')
sample_scores_w_gold

Maintenant nous savons que le marché de l'or est décorrélé du marché des actions. L'or est traditionnellement considéré comme un actif refuge en période d'incertitude économique ou de crise financière. Les investisseurs cherchent souvent à investir dans l'or lorsque les marchés boursiers connaissent des turbulences, car l'or est perçu comme une valeur refuge qui maintient sa valeur dans des conditions économiques difficiles. Il nous faut donc ajouter cette information dans notre simulation de krach et donc faire augmenter très largement les rendements de l'or (environ 40%).

In [ ]:
# Création de 2 nouvelles dates où le krach apparait
future_dates = pd.date_range(start='2024-01-13', periods=2, freq='D')  # Nouvelles dates

np.random.seed(42)
krach_returns_stocks = np.random.uniform(-0.2, -0.3, size=(2, 10))
krach_return_gold = np.random.uniform(0.4, 0.5, size=(2, 1)) 

krach_returns = pd.DataFrame(np.hstack((krach_returns_stocks, krach_return_gold)), 
                              columns=sample_returns_w_gold.columns, index=future_dates)

data_w_gold_with_krach = pd.concat([sample_returns_w_gold, krach_returns])
data_w_gold_with_krach.tail()

In [ ]:
mean_ret_krach_w_gold = data_w_gold_with_krach.mean()*252
mean_ret_krach_w_gold_dict = {ticker: round(value, 4) for ticker, value in mean_ret_krach_w_gold.items()}

optimal_weights_w_gold_esg = optimize_portfolio_krach_w_esg(sample_scores_w_gold, mean_ret_krach_w_gold_dict)
print("Pondérations optimales du portefeuille pour minimiser la perte en cas de krach avec contraintes ESG:")
for company, weight in zip(companies_names, optimal_weights_w_gold_esg):
    print(f"{company}: {weight:.4f}")

En considérant l'ESG dans notre optimisation, on peut voir que notre portefeuille est à présent uniquement composer de l'action L'Oréal, grâce à sa forte note ESG, mais également de l'or, qui voit ses rendements exploser. Si l'on optimise sans ESG notre portefeuille en simulant ce même krach, celui-ci sera quasiment uniquement composé d'or.

In [ ]:
optimal_weights_w_gold = optimize_portfolio_krach(mean_ret_krach_w_gold_dict)

print("Pondérations optimales du portefeuille pour minimiser la perte en cas de krach:")
for company, weight in zip(companies_names, optimal_weights_w_gold):
    print(f"{company}: {weight:.4f}")

Pour finir, nous allons tenter de minimser d'autre fonctions de perte pour se baser sur plusieurs métriques de risques.

In [ ]:
# Fonction pour calculer la VaR à 95%
def var_loss(weights, returns):
    weighted_returns = weights * np.array(list(returns.values()))
    loss = -np.percentile(weighted_returns, 5) 
    return loss

# Fonction pour calculer le drawdown
def drawdown_loss(weights, returns):
    wealth_index = 1000 * (1 + pd.Series(returns).cumprod())
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    max_drawdown = drawdowns.min()
    return max_drawdown

# Fonctions pour optimiser les poids du portefeuille pour minimiser la VaR et le drawdown
def optimize_portfolio_var(esg_score, returns, with_esg):
    weights_constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    esg_score_to_threshold = {
        'type' : 'eq',
        'args' : (esg_score,),
        'fun' : lambda weights, esg_score : portfolio_return(weights, esg_score) - threshold
    }
    bounds = tuple((0, 1) for _ in range(len(returns)))
    init_weights = [1 / len(returns)] * len(returns)
    if (with_esg) :
        optimal_weights = minimize(var_loss, init_weights, args=(returns,), method='SLSQP', bounds=bounds, constraints=(weights_constraints,esg_score_to_threshold))
    else : 
        optimal_weights = minimize(var_loss, init_weights, args=(returns,), method='SLSQP', bounds=bounds, constraints=weights_constraints)
    return optimal_weights.x


def optimize_portfolio_drawdown(esg_score, returns, with_esg):
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    esg_score_to_threshold = {
        'type' : 'eq',
        'args' : (esg_score,),
        'fun' : lambda weights, esg_score : portfolio_return(weights, esg_score) - threshold
    }
    bounds = tuple((0, 1) for _ in range(len(returns)))
    init_weights = [1 / len(returns)] * len(returns)
    if (with_esg) : 
        optimal_weights = minimize(drawdown_loss, init_weights, args=(returns,), method='SLSQP', bounds=bounds, constraints=(constraints, esg_score_to_threshold))
    else : 
        optimal_weights = minimize(drawdown_loss, init_weights, args=(returns,), method='SLSQP', bounds=bounds, constraints=constraints )
    return optimal_weights.x

optimal_weights_var = optimize_portfolio_var(sample_scores_w_gold, mean_ret_krach_w_gold_dict, True)
optimal_weights_drawdown = optimize_portfolio_drawdown(sample_scores_w_gold, mean_ret_krach_w_gold_dict, True)
print("Pondérations optimales du portefeuille pour minimiser le drawdown en cas de krach:")
for company, weight in zip(companies_names, optimal_weights_var):
    print(f"{company}: {weight:.4f}")


En fixant la contrainte ESG et en minimisant la Value at Risk, on peut remarquer une nouvelle composition de notre portefeuille, avec toujours l'Oréal comme composante principale.